# IBM Applied Data Science - Capstone project
## Exercises

### Topic: Predict Car Accident Severity in Seattle, USA

In [11]:
%matplotlib inline

# import libraries
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression

import os.path

In [5]:
# get the data

data_url = 'https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv'

df = pd.read_csv(data_url)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 38 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   SEVERITYCODE    194673 non-null  int64  
 1   X               189339 non-null  float64
 2   Y               189339 non-null  float64
 3   OBJECTID        194673 non-null  int64  
 4   INCKEY          194673 non-null  int64  
 5   COLDETKEY       194673 non-null  int64  
 6   REPORTNO        194673 non-null  object 
 7   STATUS          194673 non-null  object 
 8   ADDRTYPE        192747 non-null  object 
 9   INTKEY          65070 non-null   float64
 10  LOCATION        191996 non-null  object 
 11  EXCEPTRSNCODE   84811 non-null   object 
 12  EXCEPTRSNDESC   5638 non-null    object 
 13  SEVERITYCODE.1  194673 non-null  int64  
 14  SEVERITYDESC    194673 non-null  object 
 15  COLLISIONTYPE   189769 non-null  object 
 16  PERSONCOUNT     194673 non-null  int64  
 17  PEDCOUNT  

In [58]:
df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [59]:
# count unique values for a given column

id="ROADCOND"

df[id].value_counts().to_frame()

,ROADCOND
Dry,124510
Wet,47474
Unknown,15078
Ice,1209
Snow/Slush,1004
Other,132
Standing Water,115
Sand/Mud/Dirt,75
Oil,64


In [112]:
# count number of NaNs
missing_data = df.isnull()
df_nan = pd.DataFrame(columns=['ID', 'val', 'nan'])

for column in missing_data.columns.values.tolist():
    try:
        val_ok = missing_data[column].value_counts()[0]
        val_nan = missing_data[column].value_counts()[1]

        df_new_row = pd.DataFrame(data=np.array([[column,val_ok,val_nan]]), columns=['ID','val','nan'])
        df_nan = pd.concat([df_nan,df_new_row], ignore_index=True)
                               
    except:
        val_ok = missing_data[column].value_counts()[0]
        df_new_row = pd.DataFrame(data=np.array([[column,val_ok,0]]), columns=['ID','val','nan'])
        df_nan = pd.concat([df_nan,df_new_row], ignore_index=True)

# cast values as integer
df_nan = df_nan.astype({'val' :'int','nan':'int'})

df_nan.set_index('ID', inplace=True)
df_nan.sort_values(['nan'], ascending=False, axis=0, inplace=True)


df_nan
    

,val,nan
ID,,
PEDROWNOTGRNT,4667,190006
EXCEPTRSNDESC,5638,189035
SPEEDING,9333,185340
INATTENTIONIND,29805,164868
INTKEY,65070,129603
EXCEPTRSNCODE,84811,109862
SDOTCOLNUM,114936,79737
JUNCTIONTYPE,188344,6329
X,189339,5334


In [40]:
try: missing_data[column].value_counts()[1]

SyntaxError: unexpected EOF while parsing (<ipython-input-40-3d88741c1cc7>, line 1)

In [26]:
list(missing_data[column].value_counts())[0]

194673

In [62]:
missing_data[column]

0         False
1         False
2         False
3         False
4         False
          ...  
194668    False
194669    False
194670    False
194671    False
194672    False
Name: HITPARKEDCAR, Length: 194673, dtype: bool